In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np
import c3aidatalake

In [2]:
county_rec_excel = pd.ExcelFile('timeseries_USA_county_recover.xlsx')
county_rec_df = pd.read_excel(county_rec_excel, 'timeseries_USA_county_Recovered')

In [3]:
county_rec_df['county_with_naming_con'] = county_rec_df['county'].str.replace(' County', '').str.replace(' ', '')+ '_' + county_rec_df['state'].str.replace(' ','') + '_' + county_rec_df['country'].str.replace(' ','')

In [5]:
xls = pd.ExcelFile('C3-ai-Location-IDs.xlsx')
counties = pd.read_excel(xls, 'County IDs')

### Check if all the county_with_naming_con are valid

In [6]:
all(item in list(counties.iloc[:,0]) for item in list(county_rec_df['county_with_naming_con'].unique()))

True

### Check the missed counties - it should be 0

In [7]:
count = 0
for item in list(county_rec_df['county_with_naming_con'].unique()):
    if item in list(counties.iloc[:,0]):
        count += 1
    else:
        print(item)
print("{} are wrong according to the naming convention".format(len(county_rec_df['county_with_naming_con'].unique()) - count))

0 are wrong according to the naming convention


In [8]:
# Fetch facts about Germany
locations = c3aidatalake.fetch(
    "outbreaklocation",
    {
        "spec" : {
        }
    }
    , get_all = True
)

locations

,locationType,populationCDS,id,name,typeIdent,location.value.id,location.timestamp,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,...,countryCode,population2018,population2019,latestKindergartenPopulation,elementarySchoolCount,kindergartenCount,universityCount,nursingHomeCount,elderlyPopulationRatio,elderlyAloneRatio
0,state,2211875.0,'AsirRegion_SaudiArabia,"'Asir Region, SA",EP_LOC,'AsirRegion_SaudiArabia,2020-05-29T05:14:06Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,-_AndhraPradesh_India,-,EP_LOC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,00097abe-1d0c-495b-beb6-1499c1a34d47,NaN,EP_LOC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,000afd11-9970-43c5-bcd6-4c771e8be952,NaN,EP_LOC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,00112edc-92d7-48d2-a1fd-e43c37bcdb57,NaN,EP_LOC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762,NaN,NaN,zamojski_Lublin_Poland,zamojski,EP_LOC,zamojski_Lublin_Poland,2020-08-17T22:45:36Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1763,NaN,NaN,zielonogorski_Lubusz_Poland,zielonogorski,EP_LOC,zielonogorski_Lubusz_Poland,2020-08-17T22:45:34Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1764,state,3281.0,ČrnanaKoroškem_Slovenia,"Črna na Koroškem, Slovenia",EP_LOC,ČrnanaKoroškem_Slovenia,2020-05-23T20:22:40Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1765,state,10278.0,ŠmarjepriJelšah_Slovenia,"Šmarje pri Jelšah, Slovenia",EP_LOC,ŠmarjepriJelšah_Slovenia,2020-05-23T20:22:40Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pop_dict = {}

In [10]:
for county in county_rec_df['county_with_naming_con'].unique():
    if county not in pop_dict:
        pop_dict[county] = int(locations['populationCDS'].loc[locations['id']==county])

In [11]:
county_rec_df['county_population'] = county_rec_df['county_with_naming_con'].apply(lambda x:pop_dict[x])

In [12]:
county_rec_df.head()

,locationID,slug,name,level,city,county,state,country,lat,long,...,active,tested,hospitalized,hospitalized_current,discharged,icu,icu_current,date,county_with_naming_con,county_population
0,iso1:us#iso2:us-ar#fips:05001,arkansas-county-arkansas-us,"Arkansas County, Arkansas, United States",county,NaN,Arkansas County,Arkansas,United States,34.29,-91.37,...,NaN,262.0,NaN,NaN,NaN,NaN,NaN,2020-05-24,Arkansas_Arkansas_UnitedStates,17486
1,iso1:us#iso2:us-ar#fips:05001,arkansas-county-arkansas-us,"Arkansas County, Arkansas, United States",county,NaN,Arkansas County,Arkansas,United States,34.29,-91.37,...,NaN,264.0,NaN,NaN,NaN,NaN,NaN,2020-05-25,Arkansas_Arkansas_UnitedStates,17486
2,iso1:us#iso2:us-ar#fips:05001,arkansas-county-arkansas-us,"Arkansas County, Arkansas, United States",county,NaN,Arkansas County,Arkansas,United States,34.29,-91.37,...,NaN,275.0,NaN,NaN,NaN,NaN,NaN,2020-05-26,Arkansas_Arkansas_UnitedStates,17486
3,iso1:us#iso2:us-ar#fips:05001,arkansas-county-arkansas-us,"Arkansas County, Arkansas, United States",county,NaN,Arkansas County,Arkansas,United States,34.29,-91.37,...,NaN,284.0,NaN,NaN,NaN,NaN,NaN,2020-05-27,Arkansas_Arkansas_UnitedStates,17486
4,iso1:us#iso2:us-ar#fips:05001,arkansas-county-arkansas-us,"Arkansas County, Arkansas, United States",county,NaN,Arkansas County,Arkansas,United States,34.29,-91.37,...,NaN,295.0,NaN,NaN,NaN,NaN,NaN,2020-05-28,Arkansas_Arkansas_UnitedStates,17486
